In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Create Current mirror Layouts

In [2]:
k = pd.read_csv("log_files/graph.csv")
print(k)

     index  TE_0    TE_1    TE_3           Val
0        0   5.0   7.222   5.000  47209.977771
1        1   5.0   7.222   7.222  43869.303963
2        2   5.0   7.222   9.444  41651.309841
3        3   5.0   7.222  11.667  40112.485392
4        4   5.0   7.222  13.889  38858.177792
..     ...   ...     ...     ...           ...
895    895  25.0  25.000  16.111   5657.758936
896    896  25.0  25.000  18.333   4779.088961
897    897  25.0  25.000  20.556   4032.851537
898    898  25.0  25.000  22.778   3412.372544
899    899  25.0  25.000  25.000   2861.327070

[900 rows x 5 columns]


In [3]:
x = k["TE_0"].to_numpy()
y = k["TE_1"].to_numpy()
z = k["TE_3"].to_numpy()

# print(x)
f = k[("Val")].to_numpy()
f_norm = f/max(f)
# 
# plt.plot(f_norm)
# # plt.plot(x)

In [4]:
from scipy.interpolate import griddata
%matplotlib notebook

# Create a grid for x and y
xi = np.unique(x)
yi = np.unique(z)
xi, yi = np.meshgrid(xi, yi)

# Interpolate z values on the grid
zi = griddata((x, y), f_norm, (xi, yi), method='cubic')


de = [9, 2.4e+01, 2.3e+01]
fun = 324.29128151524765/max(f)


bf = [1.800e+01, 2.500e+01, 2.500e+01]
fun2 = 914.6009258666168/max(f)

# Create a surface plot
fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
ax.plot_surface(xi, yi, zi, cmap='viridis')
# ax.scatter(xi, yi, zi, color = 'blue', s=30)
ax.scatter(de[0], de[1], fun, color = 'red', s=100)
ax.scatter(bf[0], bf[1], fun2, color = 'green', s=100)

ax.set_xlabel(r'$TE_{ncm}$')
ax.set_ylabel(r'$TE_{ndp}$')
ax.set_zlabel(r'objective Func.', fontsize=12)
ax = fig.add_subplot(122, projection='3d')
ax.plot_surface(xi, yi, zi, cmap='viridis')
ax.scatter(de[1], de[2], fun, color = 'red', s=100)
ax.scatter(bf[0], bf[1], fun2, color = 'green', s=100)

ax.set_xlabel(r'$TE_{ndp}$')
ax.set_ylabel(r'$TE_{pcm}$')
ax.set_zlabel(r'objective Func.', fontsize=12)
plt.show()

<IPython.core.display.Javascript object>

In [10]:
import plotly.graph_objects as go

x = k["TE_0"].to_numpy()
y = k["TE_1"].to_numpy()
z = k["TE_3"].to_numpy()

# print(x)
f = k[("Val")].to_numpy()
f_norm = f/max(f)

# Create a grid for x and y with fewer points
xi = np.linspace(x.min(), x.max(), 50)  # 50 points instead of 100
yi = np.linspace(y.min(), y.max(), 50)  # 50 points instead of 100
xi, yi = np.meshgrid(xi, yi)

# Interpolate z values on the grid
zi = griddata((x, y), f_norm, (xi, yi), method='cubic')

# Create a Plotly heatmap
# fig = go.Figure(data=[go.Contour(z=zi, x=xi[0], y=yi[:, 0], colorscale='Viridis')])
fig = go.Figure(
    data=[go.Contour(z=zi, x=xi[0], y=yi[:, 0],  
    colorscale='Greys', 
    line=dict(color='black', width=2),  # Set contour lines to black
    showscale=False)])
fig.update_layout(
    xaxis_title='X axis',
    yaxis_title='Y axis',
    title='2D Heatmap from x, y, z Data'
)

fig.add_trace(go.Scatter(
    x=[de[0]],
    y=[de[1]],
    mode='markers+text',  # Show both markers and text
    marker=dict(size=10, color='red', symbol='circle'),
    text=[f' DE ({de[0]:.2f}, {de[1]:.2f}) <br> val = {fun}'],  # Custom text for each point
    textposition='bottom center',  # Position of the text relative to the markers
    textfont=dict(color='red'), 
    name='Data Points'

))

fig.add_trace(go.Scatter(
    x=[bf[0]],
    y=[bf[1]],
    # mode='markers',
    mode='markers+text',  # Show both markers and text
    marker=dict(size=10, color='red', symbol='circle'),
    text=[f' BFGS and SLSQP ({bf[0]:.2f}, {bf[1]:.2f}) <br> val = {fun2}'],  # Custom text for each poin
    textposition='bottom center',  # Position of the text relative to the markers
    textfont=dict(color='red'), 
    name='Data Points'

))


fig.update_layout(
    xaxis_title='X axis',
    yaxis_title='Y axis',
    title='2D Heatmap',
    width=800,   # Set the width of the figure
    height=600   # Set the height of the figure
)


# ax.scatter(de[0], de[1], fun, color = 'red', s=100)
# ax.scatter(bf[0], bf[1], fun2, color = 'green', s=100)
# Show the plot
fig.show()
fig.write_image("cont.svg")

Low voltage Cascode Current Mirror Test

In [15]:
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline, interp1d
from scipy.misc import derivative
from LUT.fetch_ import *
from tools.log_file import *
from sklearn.linear_model import LinearRegression
import time
from circuit_design.common_functions import *

VGS = np.linspace(0, 0.8, 41)
VDS = np.linspace(0, 0.8, 41)
VSB = np.array([0])
LEN = np.array([14e-9])
FIN = np.array([2, 6, 10, 20, 40])
v_margin = 0.05

tb = fetch_data_1()  # tb is for table

In [19]:
gm = tb.lookup("gm", 1, 10, type, vds_val=0.4)
ids = tb.lookup("id", 1, 10, type, vds_val=0.4)
if type == "p":
    ids = -1*ids
gm_id_ = gm/ids
vgs = interpolate_array(VGS, gm_id_, 16)
print(vgs)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'NoneType'

In [17]:
# M5 and M4 (LOW VOLTAGE CASCODE) -----------------------------------------------
vz = 1.2/3
vds_5 = vz
fin_ref = 10

gm_id_4 = 10.87
gm_id_5 = 9.65

for _ in range(5):
    gm_5_ = tb.lookup("gm", 1, fin_ref, "n", vds_val=vds_5)
    id_5_ = tb.lookup("id", 1, fin_ref, "n", vds_val=vds_5)
    gm_id_5_ = gm_5_ / id_5_
    vgs_5 = interpolate_array(VGS, gm_id_5_, gm_id_5)
    vw = vgs_5
    vz = vw - 2/gm_id_4 + v_margin
    vds_4 = vw - vz
    gm_4_ = tb.lookup("gm", 1, 20, "n", vds_val=vds_4)
    id_4_ = tb.lookup("id", 1, 20, "n", vds_val=vds_4)
    gm_id_4_ = gm_4_ / id_4_
    vgs_4 = interpolate_array(VGS, gm_id_4_, 
                              gm_id_4)
    vb3 = vz + vgs_4
    
print(vz, vw)
print(vb3)

0.3633046570029091 0.4972972972972973
0.826167519865772
